In [1]:
import pandas as pd
import numpy as np
import random

from tqdm.notebook import tqdm

In [2]:
from geopy.distance import geodesic

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("London postcodes.csv", delimiter=',')

In [5]:
df.shape

(323306, 49)

In [6]:
df.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,Nearest station,Distance to station,Postcode area,Postcode district,Police force,Water company,Plus Code,Average Income,Sewage Company,Travel To Work Area
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,...,Bromley South,0.218257,BR,BR1,Metropolitan Police,Thames Water,9F32C228+J5,63100,NaN,London
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,...,Bromley North,0.253666,BR,BR1,Metropolitan Police,Thames Water,9F32C248+G3,56100,NaN,London
2,BR1 1AD,No,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,...,Bromley South,0.044559,BR,BR1,Metropolitan Police,NaN,9F32C228+2M,63100,NaN,London
3,BR1 1AE,Yes,51.404543,0.014195,540197,169204,TQ401692,Greater London,Bromley,Bromley Town,...,Bromley North,0.462939,BR,BR1,Metropolitan Police,Thames Water,9F32C237+RM,63100,NaN,London
4,BR1 1AF,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,...,Bromley South,0.227664,BR,BR1,Metropolitan Police,Thames Water,9F32C227+HX,63100,NaN,London


In [7]:
df.columns

Index(['Postcode', 'In Use?', 'Latitude', 'Longitude', 'Easting', 'Northing',
       'Grid Ref', 'County', 'District', 'Ward', 'District Code', 'Ward Code',
       'Country', 'County Code', 'Constituency', 'Introduced', 'Terminated',
       'Parish', 'National Park', 'Population', 'Households', 'Built up area',
       'Built up sub-division', 'Lower layer super output area', 'Rural/urban',
       'Region', 'Altitude', 'London zone', 'LSOA Code', 'Local authority',
       'MSOA Code', 'Middle layer super output area', 'Parish Code',
       'Census output area', 'Constituency Code',
       'Index of Multiple Deprivation', 'Quality', 'User Type', 'Last updated',
       'Nearest station', 'Distance to station', 'Postcode area',
       'Postcode district', 'Police force', 'Water company', 'Plus Code',
       'Average Income', 'Sewage Company', 'Travel To Work Area'],
      dtype='object')

In [8]:
rides = pd.DataFrame(columns=['driver_id', 'client_id',\
                              'start', 'start_latitude', 'start_longtitude', \
                              'finish', 'finish_latitude', 'finish_longtitude', \
                              'distance', 'road_time', 'start_time', 'finish_time', 'cost', \
                              'driver_rate', 'category_driver_feedback', 'text_driver_feedback',\
                             'client_rate', 'category_client_feedback', 'text_client_feedback'])
NUM_RIDES = 5000000

Drivers and clients id's

In [9]:
rides['driver_id'] = np.random.randint(low=0, high=2500, size=NUM_RIDES)
rides['client_id'] = np.random.randint(low=0, high=4500, size=NUM_RIDES)

Start and finish points

In [10]:
rides[['start', 'start_latitude', 'start_longtitude']] = df[['Postcode', 'Latitude', 'Longitude']].sample(n=NUM_RIDES, replace=True).reset_index(drop=True)

In [11]:
rides[['finish', 'finish_latitude', 'finish_longtitude']] = df[['Postcode', 'Latitude', 'Longitude']].sample(n=NUM_RIDES, replace=True).reset_index(drop=True)

Start time

In [12]:
def random_dates(start, end, n=10):
    start_u = start.value//10**9
    end_u = end.value//10**9
    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

start = pd.to_datetime('2010-01-01')
end = pd.to_datetime('2020-01-01')
rides['start_time'] = random_dates(start, end, NUM_RIDES)

Distance between start and finish points

In [13]:
rides['distance'] = [geodesic((x1, y1), (x2, y2)).km for x1, y1, x2, y2 in tqdm(zip(rides['start_latitude'], \
                                                                                              rides['start_longtitude'], \
                                                                                              rides['finish_latitude'], \
                                                                                              rides['finish_longtitude']), total=NUM_RIDES)]
rides['distance'] = rides['distance'].round(2)

Calculate road time

In [14]:
rides['road_time'] = abs(np.random.normal(size=NUM_RIDES, scale=10)) + rides['distance'] * abs(np.random.normal(size=NUM_RIDES, loc=1, scale=0.25))
rides['road_time'] = rides['road_time'].astype('int')
rides['road_time'] = pd.to_timedelta(rides['road_time'], unit='m')

Calculate finish time

In [15]:
rides['finish_time'] = rides['start_time'] + rides['road_time']

Calculate cost of the ride

In [16]:
def count_cost(start_time, distance):
    cost = 2 + 0.5 * distance
    if (start_time.hour >= 8 and start_time.hour <= 9) or \
        (start_time.hour >= 18 and start_time.hour <= 19):
        cost *= 1.5
    if (start_time.hour >= 22 or start_time.hour <= 6):
        cost *= 1.3
    return cost
    
rides['cost'] = [count_cost(s, d) for s, d in tqdm(zip(rides.start_time, rides.distance), total=NUM_RIDES)]
rides['cost'] = rides['cost'].round(2)

Drivers rates

In [17]:
driver_rate_idx = np.random.randint(low=0, high=NUM_RIDES, size=int(NUM_RIDES*0.3))
driver_rate_distribution_arr = np.random.multinomial(1, [0.2, 0.05, 0.1, 0.25, 0.4], size=int(NUM_RIDES*0.3))
rides['driver_rate'][driver_rate_idx] = np.where(driver_rate_distribution_arr == 1)[1] + 1

In [18]:
driver_feedback_categories_good = ['great service', 'nice car', 'wonderful companion', 'neat and tidy', 'expert navigation', 'recommend']
driver_feedback_categories_bad = ['awful service', 'bad car', 'unpleasant companion', 'dirty', 'non-expert navigation', 'not recommend']

In [19]:
category_driver_good_feedback_idx = np.random.choice(rides[rides.driver_rate > 3].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_driver_feedback"][category_driver_good_feedback_idx] = np.random.choice(driver_feedback_categories_good, size=int(NUM_RIDES*0.3*0.2))

category_driver_bad_feedback_idx = np.random.choice(rides[rides.driver_rate < 4].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_driver_feedback"][category_driver_bad_feedback_idx] = np.random.choice(driver_feedback_categories_bad, size=int(NUM_RIDES*0.3*0.2))

In [20]:
text_good_feedback_driver_length = np.random.randint(low=0, high=7, size=int(NUM_RIDES*0.3*0.2))
text_good_feedback_driver_sample = [random.sample(driver_feedback_categories_good, i) for i in text_good_feedback_driver_length]
rides['text_driver_feedback'][category_driver_good_feedback_idx] = text_good_feedback_driver_sample

text_bad_feedback_driver_length = np.random.randint(low=0, high=7, size=int(NUM_RIDES*0.3*0.2))
text_bad_feedback_driver_sample = [random.sample(driver_feedback_categories_bad, i) for i in text_bad_feedback_driver_length]
rides['text_driver_feedback'][category_driver_bad_feedback_idx] = text_bad_feedback_driver_sample

Clients rates

In [21]:
client_rate_idx = np.random.randint(low=0, high=NUM_RIDES, size=int(NUM_RIDES*0.5))
client_rate_distribution_arr = np.random.multinomial(1, [0.2, 0.05, 0.1, 0.25, 0.4], size=int(NUM_RIDES*0.5))
rides['client_rate'][client_rate_idx] = np.where(client_rate_distribution_arr == 1)[1] + 1

In [22]:
client_feedback_categories_good = ['polite', 'pleasant', 'quiet', 'neat and tidy', 'recommend']
client_feedback_categories_bad = ['unpolite', 'unpleasant', 'loud', 'dirty','not recommend']

In [23]:
category_client_good_feedback_idx = np.random.choice(rides[rides.client_rate > 3].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_client_feedback"][category_client_good_feedback_idx] = np.random.choice(client_feedback_categories_good, size=int(NUM_RIDES*0.3*0.2))

category_client_bad_feedback_idx = np.random.choice(rides[rides.client_rate < 4].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_client_feedback"][category_client_bad_feedback_idx] = np.random.choice(client_feedback_categories_bad, size=int(NUM_RIDES*0.3*0.2))

In [24]:
text_good_feedback_client_length = np.random.randint(low=0, high=6, size=int(NUM_RIDES*0.3*0.2))
text_good_feedback_client_sample = [random.sample(client_feedback_categories_good, i) for i in text_good_feedback_client_length]
rides['text_client_feedback'][category_client_good_feedback_idx] = text_good_feedback_client_sample

text_bad_feedback_client_length = np.random.randint(low=0, high=6, size=int(NUM_RIDES*0.3*0.2))
text_bad_feedback_client_sample = [random.sample(client_feedback_categories_good, i) for i in text_bad_feedback_client_length]
rides['text_client_feedback'][category_client_good_feedback_idx] = text_bad_feedback_client_sample

In [25]:
rides.head()

,driver_id,client_id,start,start_latitude,start_longtitude,finish,finish_latitude,finish_longtitude,distance,road_time,start_time,finish_time,cost,driver_rate,category_driver_feedback,text_driver_feedback,client_rate,category_client_feedback,text_client_feedback
0,1712,4304,N7 9BF,51.547105,-0.120730,SE10 9GH,51.481154,-0.011670,10.54,0 days 00:08:00,2014-09-30 10:34:14,2014-09-30 10:42:14,7.27,NaN,NaN,NaN,5,NaN,NaN
1,1355,3004,HA7 3NF,51.615673,-0.331458,CR4 1NR,51.412686,-0.148536,25.91,0 days 00:21:00,2012-12-15 06:44:11,2012-12-15 07:05:11,19.44,NaN,NaN,NaN,NaN,NaN,NaN
2,1919,3797,SM6 0XX,51.360296,-0.150894,N1 8XL,51.533075,-0.105534,19.48,0 days 00:23:00,2011-08-28 23:24:23,2011-08-28 23:47:23,15.26,NaN,NaN,NaN,4,NaN,NaN
3,1438,4477,SE13 7XT,51.459240,-0.013354,BR3 2HL,51.409991,-0.018955,5.49,0 days 00:19:00,2017-06-14 01:18:15,2017-06-14 01:37:15,6.17,NaN,NaN,NaN,NaN,NaN,NaN
4,1930,1618,SW2 4PG,51.440297,-0.135707,IG2 6QA,51.575109,0.079599,21.18,0 days 00:17:00,2019-05-29 02:51:19,2019-05-29 03:08:19,16.37,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
rides.to_csv("rides.csv")